step 1: If we need to consume a local build of [dotnet/interactive](https://github.com/dotnet/interactive) repo, run the following cell

In [ ]:
$dotnetInteractivePath="D:\microsoft\dotnet\interactive"
if ($IsWindows) {
    Invoke-Expression "$dotnetInteractivePath\build-js.cmd"
}else {
    Invoke-Expression "$dotnetInteractivePath\build-js.sh"
}

step 2: Create the NuGet package CSharpProject

In [ ]:
Invoke-Expression "$dotnetInteractivePath\repack.ps1"

step 3: The previous step copied the generated packages into from C:\temp\packages\
       Create a new NuGet Feed pointing to this path

step 4: Build of the JavaScript code in [trydotnet service](https://github.com/dotnet/try) repo

In [1]:
$tryDotnetPath="D:\microsoft\dotnet\try"
if ($IsWindows) {
    Invoke-Expression "$tryDotnetPath\build-js.cmd"
}else {
    Invoke-Expression "$tryDotnetPath\build-js.sh"
}

Building NPM in directory src\microsoft-trydotnet


Error: Command cancelled.

# step 5: Build of trydotnet service

In [ ]:
Invoke-Expression "$tryDotnet\build.cmd"

step 6: Open try dotnet in Visual Studio and set Microsoft.TryDotNet as a startup project

step 7: Run try dotnet (F5)

step 8: code D:\microsoft\dotnet\try\src\microsoft-learn-mock and then open menu Terminal and start a new terminal
     

step 9: 
npm ci
npm run buildProd
npx http-server

http://192.168.1.68:8080/site/DocsHost.html?trydotnetUrl=https%3A%2F%2Flocalhost%3A7061%2Fapi%2Ftrydotnet.min.js&trydotnetOrigin=https%3A%2F%2Flocalhost%3A7061%2F

step 9: in vscode, go to menu Terminal and then new Termianlnpx http-server

In [50]:
sequenceDiagram
    box Browser
    participant U as user browser
    participant I as trydotnet.microsoft.com iframe
    end
    box Server
    participant A as learn.microsoft.com
    participant B as trydotnet.microsoft.com
    participant C as CSharpProject:Backend
    end
    U->>A: request to open the page
    A->>U: page containing iframe, trydotnet.js links
    U->>A: request trydotnet.js
    A-->>U: trydotnet.js
    I->>B: request trydotnet.microsoft.com/editor
    B-->>I: monaco editor
    U->>B: GET /editor: iFrame load c sharp text editor


